<a href="https://colab.research.google.com/github/TheLemonPig/RL-SSM/blob/main/rlwm_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'pymc>=5.9'
!pip install numpyro

In [1]:
import pymc as pm
import numpy as np
import random, pickle
import matplotlib.pyplot as plt
import pandas as pd
import pytensor
import pytensor.tensor as pt

##PyTensor

In [129]:
# FUNCTIONS

# def rlwm_step(dR, pA, pG, pP, dq_RL_tm1, dq_WM_tm1):
#     cond = pt.switch((dR - dq_RL_tm1) >= 0, 1, 0)
#     dq_RL_t0 = dq_RL_tm1 + (cond + (1.0 - cond) * pG) * pA * (dR - dq_RL_tm1)
#     dq_WM_t0 = dq_WM_tm1 + (cond + (1.0 - cond) * pG) * 1.0 * (dR - dq_RL_tm1)
#     dq_WM_t0 = dq_WM_t0 + pP * ((1 / dR.shape[2]) - dq_WM_t0)
#     return [dq_RL_t0, dq_WM_t0]

def rlwm_step(dC, dR, pA, pG, pP, dq_RL, dq_WM):
    cond = pt.switch((dR - dq_RL) >= 0, 1, 0)
    dq_RL += (cond + (1.0 - cond) * pG) * pA * (dR - dq_RL) * dC
    dq_WM += (cond + (1.0 - cond) * pG) * 1.0 * (dR - dq_RL) * dC
    dq_WM += pP * ((1 / dR.shape[2]) - dq_WM)
    return [dq_RL, dq_WM]

# def rlwm_step(dC, dR, pA, pG, pP, mask, cond, dQ):
#     dQ += (cond + (1.0 - cond) * pG) * pA * (dR - dQ) * dC
#     dQ += pP * ((1 / dR.shape[2]) - dQ) * mask
#     return dQ

# def rlwm_scan(dC, dR, pA, pG, pP, dq_RL, dq_WM):
#     shape = dC.shape
#     new_shape = (shape[0], shape[1], shape[2], shape[3], shape[4], 1)
#     mask = pt.stack([pt.zeros(shape),pt.ones(shape)],axis=-1)  # currently mask covers RL Q-values
#     cond = pt.switch((dR - dq_RL) >= 0, 1, 0)
#     cond = pt.repeat(cond.reshape(new_shape), 2, axis=5)
#     dC = pt.repeat(dC.reshape(new_shape), 2, axis=5)
#     dR = pt.repeat(dR.reshape(new_shape), 2, axis=5)
#     pA = pt.stack([pA, pt.ones_like(pA)], axis=-1) # This sets the alpha values for Q_WM to 1.0
#     pG = pt.repeat(pG.reshape(new_shape), 2, axis=5)
#     pP = pt.repeat(pP.reshape(new_shape), 2, axis=5)
#     dQ = pt.stack([dq_RL, dq_WM],axis=-1)
#     dQ, _ = pytensor.scan(rlwm_step, sequences=[dC, dR, pA, pG, pP, mask, cond], non_sequences=[], outputs_info=[dQ])
#     return dQ

# TODO: Combine dq_RL and dq_WM into one matrix and use a mask function to select one

# def rlwm_step(dC, dR, pA, pG, pP, dQ_tm1):
#     shape = dR.shape
#     cond = pt.switch((dR - dq_RL_tm1) >= 0, 1, 0)
#     dq_RL_t0 = dq_RL_tm1 + (cond + (1.0 - cond) * pG) * pA * (dR - dq_RL_tm1) * dC
#     dq_WM_t0 = dq_WM_tm1 + (cond + (1.0 - cond) * pG) * 1.0 * (dR - dq_WM_tm1) * dC
#     dq_WM_t0 = dq_WM_t0 + pP * ((1 / dR.shape[2]) - dq_WM_t0)
#     # dQ = dQ + pt.concat([dq_RL_t0.reshape((shape[0], shape[1], shape[2], shape[3], shape[4], 1)), dq_WM_t0.reshape((shape[0], shape[1], shape[2], shape[3], shape[4], 1))], axis=5)
#     dQ_t0 = dQ_tm1 + pt.stack([dq_RL_t0, dq_WM_t0], axis=-1)
#     return dQ_t0

# def rlwm_scan(dC, dR, pA, pG, pP, dq_RL, dq_WM):
#     shape = dC.shape
#     dQ = pt.stack([dq_RL, dq_WM], axis=-1)
#     ([dq_RL, dq_WM], _) = pytensor.scan(rlwm_step, sequences=[dC, dR, pA, pG, pP], non_sequences=[], outputs_info=[dQ])
#     return dq_RL, dq_WM

def rlwm_scan(dC, dR, pA, pG, pP, dq_RL, dq_WM):
    ([dq_RL, dq_WM], _) = pytensor.scan(rlwm_step, sequences=[dC, dR, pA, pG, pP], non_sequences=[], outputs_info=[dq_RL, dq_WM])
    return dq_RL, dq_WM


def rlwm_softmax(Qs, pB):
    shape = Qs.shape
    tempered_qs = Qs * pB
    qs_max = tempered_qs.max(axis=2)
    qs_max = pt.repeat(qs_max.reshape((shape[0], shape[1], 1, shape[3], shape[4])), shape[2], axis=2)
    numerator = pt.exp(tempered_qs - qs_max)
    denominator = numerator.sum(axis=2)
    denominator = pt.repeat(denominator.reshape((shape[0], shape[1], 1, shape[3], shape[4])), shape[2], axis=2)
    Ps = numerator / denominator
    return Ps


def rlwm_likelihood(dC, dq_RL, dq_WM, pB, pC, pE, pR, set_sizes):
    weight = pR * pt.clip(pC / set_sizes, -np.inf, 1)
    Ps_RL = rlwm_softmax(dq_RL, pB)
    Ps_WM = rlwm_softmax(dq_WM, pB)
    pol = weight * Ps_WM + (1.0 - weight) * Ps_RL
    pol_final = (1.0 - pE) * pol + pE * (1.0 / dC.shape[2])
    likelihood = pt.log(pol_final)
    return likelihood

def rlwm_Ps(dq_RL, dq_WM, dC, dR, pA, pB, pG, pP):
    dq_RL, dq_WM = rlwm_scan(dC, dR, pA, pG, pP, dq_RL, dq_WM)
    Ps_RL = rlwm_softmax(dq_RL, pB)
    Ps_WM = rlwm_softmax(dq_WM, pB)
    return Ps_RL, Ps_WM


def rlwm_recovery(dq_RL, dq_WM, dC, dR, pA, pB, pC, pE, pG, pP, pR, set_sizes):
    dq_RL, dq_WM = rlwm_scan(dC, dR, pA, pG, pP, dq_RL, dq_WM)
    likelihood = rlwm_likelihood(dC, dq_RL, dq_WM, pB, pC, pE, pR, set_sizes)
    return likelihood

# COMPILERS

def rlwm_step_compile():
    dC4 = pt.dtensor4("dC4")
    dR4 = pt.dtensor4("dR4")
    dq_RL4 = pt.dtensor4("dq_RL4")
    dq_WM4 = pt.dtensor4("dq_WM4")
    pA4 = pt.dtensor4("pA4")
    pG4 = pt.dtensor4("pG4")
    pP4 = pt.dtensor4("pP4")

    dq_RL, dq_WM = rlwm_step(dC4, dR4, pA4, pG4, pP4, dq_RL4, dq_WM4)
    rlwm_step_func = pytensor.function(inputs=[dC4, dR4, pA4, pG4, pP4, dq_RL4, dq_WM4], outputs=[dq_RL, dq_WM])

    return rlwm_step_func

# def rlwm_step_compile():
#     dC5 = pt.dtensor5("dC5")
#     dR5 = pt.dtensor5("dR5")
#     pA5 = pt.dtensor5("pA5")
#     pG5 = pt.dtensor5("pG5")
#     pP5 = pt.dtensor5("pP5")
#     mask = pt.dtensor5("mask")
#     cond = pt.dtensor5("cond")
#     dQ5 = pt.dtensor5("dQ5")

#     dQ = rlwm_step(dC5, dR5, pA5, pG5, pP5, mask, cond, dQ5)

#     rlwm_step_func = pytensor.function(inputs=[dC5, dR5, pA5, pG5, pP5, mask, cond, dQ5], outputs=[dQ])

#     return rlwm_step_func


def rlwm_scan_compile():
    dC5 = pt.dtensor5("dC5")
    dR5 = pt.dtensor5("dR5")
    dq_RL4 = pt.dtensor4("dq_RL4")
    dq_WM4 = pt.dtensor4("dq_WM4")
    pA5 = pt.dtensor5("pA5")
    pG5 = pt.dtensor5("pG5")
    pP5 = pt.dtensor5("pP5")

    # dQ = rlwm_scan(dC5, dR5, pA5, pG5, pP5, dq_RL4, dq_WM4)
    # rlwm_step_func = pytensor.function(inputs=[dC5, dR5, pA5, pG5, pP5, dq_RL4, dq_WM4], outputs=[dQ])
    dq_RL, dq_WM = rlwm_scan(dC5, dR5, pA5, pG5, pP5, dq_RL4, dq_WM4)
    rlwm_step_func = pytensor.function(inputs=[dC5, dR5, pA5, pG5, pP5, dq_RL4, dq_WM4], outputs=[dq_RL, dq_WM])

    return rlwm_step_func


def rlwm_softmax_compile():
    Qs = pt.dtensor5('Qs')
    B = pt.dtensor5('B')

    Ps = rlwm_softmax(Qs, B)
    Ps_func = pytensor.function(inputs=[Qs, B], outputs=[Ps])

    return Ps_func


def rlwm_likelihood_compile():
    dC5 = pt.dtensor5("dC5")
    dq_RL5 = pt.dtensor5("dq_RL5")
    dq_WM5 = pt.dtensor5("dq_WM5")
    pB5 = pt.dtensor5("pB5")
    pC5 = pt.dtensor5("pC5")
    pE5 = pt.dtensor5("pE5")
    pR5 = pt.dtensor5("pR5")
    set_sizes = pt.dtensor5("set_sizes")

    likelihood = rlwm_likelihood(dC5, dq_RL5, dq_WM5, pB5, pC5, pE5, pR5, set_sizes)
    rlwm_likelihood_func = pytensor.function(inputs=[dC5, dq_RL5, dq_WM5, pB5, pC5, pE5, pR5, set_sizes], outputs=[likelihood])

    return rlwm_likelihood_func

def rlwm_Ps_compile():

    dq_RL = pt.dtensor4("dq_RL")
    dq_WM = pt.dtensor4("dq_WM")
    dC = pt.dtensor5("dC")
    dR = pt.dtensor5("dR")
    pA = pt.dtensor5("pA")
    pB = pt.dtensor5("pB")
    pG = pt.dtensor5("pG")
    pP = pt.dtensor5("pP")

    Ps_RL, Ps_WM = rlwm_Ps(dq_RL, dq_WM, dC, dR, pA, pB, pG, pP)
    rlwm_Ps_func = pytensor.function(inputs=[dq_RL, dq_WM, dC, dR, pA, pB, pG, pP], outputs=[Ps_RL, Ps_WM])

    return rlwm_Ps_func

def rlwm_recovery_compile():
    dq_RL = pt.dtensor4("dq_RL")
    dq_WM = pt.dtensor4("dq_WM")
    dC = pt.dtensor5("dC")
    dR = pt.dtensor5("dR")
    pA = pt.dtensor5("pA")
    pB = pt.dtensor5("pB")
    pC = pt.dtensor5("pC")
    pE = pt.dtensor5("pE")
    pG = pt.dtensor5("pG")
    pP = pt.dtensor5("pP")
    pR = pt.dtensor5("pR")
    set_sizes = pt.dtensor5("set_sizes")

    likelihood = rlwm_recovery(dq_RL, dq_WM, dC, dR, pA, pB, pC, pE, pG, pP, pR, set_sizes)
    rlwm_recovery_func = pytensor.function(inputs=[dq_RL, dq_WM, dC, dR, pA, pB, pC, pE, pG, pP, pR, set_sizes], outputs=[likelihood])

    return rlwm_recovery_func

# TESTS

def rlwm_step_test():
    n_participants = 6
    n_choices = 3
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    #shape = (n_participants, n_choices, max_stimuli, n_blocks, 2)
    shape = (n_participants, n_choices, max_stimuli, n_blocks)

    dC5_ = np.random.randint(low=0, high=n_choices, size=shape)
    dR5_ = np.random.randint(low=0, high=1, size=shape)
    pA5_ = np.ones(shape) * 0.1
    pG5_ = np.ones(shape) * 0.9
    pP5_ = np.ones(shape) * 1.0
    mask = np.ones(shape)
    cond = np.ones(shape)
    dQ5_ = np.ones(shape) * 1/n_choices
    dq_RL4_ = np.ones(shape) * 1/n_choices
    dq_WM4_ = np.ones(shape) * 1/n_choices

    test_func = rlwm_step_compile()

    # return test_func(dC5_, dR5_, pA5_, pG5_, pP5_, mask, cond, dQ5_)
    return test_func(dC5_, dR5_, pA5_, pG5_, pP5_, dq_RL4_, dq_WM4_)


def rlwm_scan_test():
    n_trials = 23
    n_participants = 6
    n_choices = 3
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape5 = (n_trials, n_participants, n_choices, max_stimuli, n_blocks)

    dC5_ = np.random.randint(low=0, high=n_choices, size=shape5)
    dR5_ = np.random.randint(low=0, high=1, size=shape5)
    dq_RL4_ = np.ones_like(dR5_)[0] * 1 / n_choices
    dq_WM4_ = np.ones_like(dR5_)[0] * 1 / n_choices
    pA5_ = np.ones_like(dR5_) * 0.1
    pG5_ = np.ones_like(dR5_) * 0.9
    pP5_ = np.ones_like(dR5_) * 1.0

    test_func = rlwm_scan_compile()

    return test_func(dC5_, dR5_, pA5_, pG5_, pP5_, dq_RL4_, dq_WM4_)


def rlwm_softmax_test():
    n_trials = 23
    n_participants = 6
    n_choices = 3
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape = (n_trials, n_participants, n_choices, max_stimuli, n_blocks)

    Qs = np.ones(shape)
    B = np.ones_like(Qs)

    test_func = rlwm_softmax_compile()

    return test_func(Qs, B)


def rlwm_likelihood_test():
    n_trials = 23
    n_participants = 6
    n_choices = 3
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape5 = (n_trials, n_participants, n_choices, max_stimuli, n_blocks)

    dC5_ = np.random.randint(low=0, high=n_choices, size=shape5)
    dq_RL5_ = - np.ones_like(dC5_) * 1 / n_choices
    dq_WM5_ = - np.ones_like(dC5_) * 1 / n_choices
    pB5_ = np.ones_like(dC5_) * 1.0
    pC5_ = np.ones_like(dC5_) * 4.0
    pE5_ = np.ones_like(dC5_) * 0.5
    pR5_ = np.ones_like(dC5_) * 0.8
    set_sizes_ = np.tile(n_stimuli, (shape5[0], shape5[1], shape5[2], shape5[3], 1))

    test_func = rlwm_likelihood_compile()

    return test_func(dC5_, dq_RL5_, dq_WM5_, pB5_, pC5_, pE5_, pR5_, set_sizes_)


def rlwm_Ps_test():
    n_trials = 23
    n_participants = 7
    n_choices = 4
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape = (n_trials, n_participants, n_choices, max_stimuli, n_blocks)

    dq_RL_ = np.ones(shape)[0] * 1.0 / n_choices
    dq_WM_ = np.ones(shape)[0] * 1.0 / n_choices

    dC_ = np.random.randint(low=0, high=n_choices, size=shape)
    dR_ = np.random.randint(low=0, high=1, size=shape)
    pA_ = np.ones_like(dC_) * 0.01
    pB_ = np.ones_like(dC_) * 1.0
    pG_ = np.ones_like(dC_) * 0.8
    pP_ = np.ones_like(dC_) * 0.3

    test_func = rlwm_Ps_compile()

    return test_func(dq_RL_, dq_WM_, dC_, dR_, pA_, pB_, pG_, pP_)


def rlwm_recovery_test():
    n_trials = 23
    n_participants = 7
    n_choices = 4
    n_stimuli = [4, 5, 6, 7, 8]
    max_stimuli = max(n_stimuli)
    n_blocks = 5
    shape = (n_trials, n_participants, n_choices, max_stimuli, n_blocks)

    dq_RL_ = np.ones(shape)[0] * 1.0 / n_choices
    dq_WM_ = np.ones(shape)[0] * 1.0 / n_choices

    dC_ = np.random.randint(low=0, high=n_choices, size=shape)
    dR_ = np.random.randint(low=0, high=1, size=shape)
    pA_ = np.ones_like(dC_) * 0.01
    pB_ = np.ones_like(dC_) * 1.0
    pC_ = np.ones_like(dC_) * 4.0
    pE_ = np.ones_like(dC_) * 0.02
    pG_ = np.ones_like(dC_) * 0.8
    pP_ = np.ones_like(dC_) * 0.3
    pR_ = np.ones_like(dC_) * 0.8
    set_sizes_ = np.tile(n_stimuli, (shape[0], shape[1], shape[2], shape[3], 1))

    test_func = rlwm_recovery_compile()

    return test_func(dq_RL_, dq_WM_, dC_, dR_, pA_, pB_, pC_, pE_, pG_, pP_, pR_, set_sizes_)

In [113]:
rlwm_step_compile()
rlwm_step_test()[0].shape, rlwm_step_test()[1].shape

((6, 3, 8, 5), (6, 3, 8, 5))

In [116]:
rlwm_scan_compile()
# dq_rl, dq_wm = rlwm_scan_test()
# dq_rl.shape, dq_wm.shape
dQ = rlwm_scan_test()
dQ[0].shape

(23, 6, 3, 8, 5)

In [ ]:
rlwm_softmax_compile()
rlwm_softmax_test()

In [ ]:
rlwm_likelihood_compile()
rlwm_likelihood_test()

In [ ]:
rlwm_Ps_compile()
rlwm_Ps_test()

In [ ]:
rlwm_recovery_compile()
rlwm_recovery_test()

##RLWM Data Generation

In [131]:
model_config_rl = {
    "RLWM_v2": {
        "doc": "RLWM model v2.",
        "params": ["rl_alpha", "rl_phi", "rl_rho", "rl_gamma", "rl_epsilon", "C"],
        "param_bounds": [[0.0, 0.0, 0.0, 0.0, 0.0, 2.0], [1.0, 1.0, 1.0, 1.0, 1.0, 5.0]],
    }
}

In [132]:
def gen_trial_seq(set_size, num_rep_per_block):
    trials = list()

    for stim in np.arange(set_size):
        trials.extend([stim]*random.choice(num_rep_per_block))
    random.shuffle(trials)

    return trials

def gen_SR_map(set_size, num_actions):
    S_R_map = {}
    acts = np.arange(num_actions)

    for stim in np.arange(set_size):
        S_R_map[stim] = random.choice(acts)

    return S_R_map

def step_action(s, a, S_R_map):
    if a == S_R_map[s]:
        rew = 1
    else:
        rew = 0

    return rew

def softmax(q_val, beta):
    q_val = np.array(q_val)*beta
    q_val = q_val - np.max(q_val)
    q_val = np.exp(q_val)
    q_val = q_val / np.sum(q_val)
    return q_val

In [133]:
def simulate_RLWM(param_rl, set_size_list, num_rep_per_block, num_actions, subjs=1):

    sub_list = np.arange(subjs)
    actions = np.arange(num_actions)

    sub_list_sub_id = list()
    sub_list_block_num = list()
    sub_list_stim = list()
    sub_list_actions = list()
    sub_list_reward = list()
    sub_list_corr = list()
    sub_list_stim_ctr = list()
    sub_list_set_size = list()

    for subj in sub_list:
        # RLWM parameters
        alpha = param_rl[0] # RL learning rate
        phi = param_rl[1] # WM decay
        rho = param_rl[2] # WM weight
        gamma = param_rl[3] # perseveration on negative outcomes
        epsilon = param_rl[4] # weight of uniform policy
        C = param_rl[5] # WM capacity
        beta = 100 # inverse temperature

        pol = np.zeros(num_actions)

        for bl in range(len(set_size_list)):
            #print(">> block -- ", bl)
            set_size = set_size_list[bl]

            S_R_map = gen_SR_map(set_size, num_actions)
            trials = gen_trial_seq(set_size, num_rep_per_block)

            # print(trials)

            q_RL = np.ones((set_size, num_actions)) * 1/num_actions
            q_WM = np.ones((set_size, num_actions)) * 1/num_actions
            weight = rho * min(1, C/set_size)

            stim_counter = np.zeros(set_size)

            for tr in np.arange(len(trials)):
                state = trials[tr]
                stim_counter[state] += 1

                pol_RL = softmax(q_RL[state, :], beta)
                pol_WM = softmax(q_WM[state, :], beta)

                pol = weight * pol_WM + (1-weight) * pol_RL

                pol_final = (1 - epsilon) * pol + epsilon * np.tile([1/num_actions], num_actions)

                action = np.random.choice(actions, p=pol_final)


                reward = step_action(state, action, S_R_map)
                #print("\t\t\tdone action", state, action, reward)

                #print("\t\t\tupdating q")
                if reward == 1:
                    sub_list_corr.append(1)
                    q_RL[state, action] = q_RL[state, action] + alpha * (reward - q_RL[state, action])
                    q_WM[state, action] = reward
                elif reward == 0:
                    sub_list_corr.append(0)
                    q_RL[state, action] = q_RL[state, action] + gamma * alpha * (reward - q_RL[state, action])
                    q_WM[state, action] = q_WM[state, action] + gamma * (reward - q_WM[state, action])
                #print("\t\t\tdone updating q")
                q_WM = q_WM + phi * ((1/num_actions)-q_WM)
                #print("\t\t\tdone WM decay")

                # store data
                sub_list_sub_id.append(subj)
                sub_list_block_num.append(bl)
                sub_list_stim.append(state)
                sub_list_actions.append(action)
                sub_list_reward.append(reward)
                sub_list_stim_ctr.append(stim_counter[state])
                sub_list_set_size.append(set_size)
        #     print("\t\t -- end trial")
        # print("\t -- end block")

    sub_list_sub_id = np.array(sub_list_sub_id)
    sub_list_stim = np.array(sub_list_stim)
    sub_list_actions = np.array(sub_list_actions)
    sub_list_reward = np.array(sub_list_reward)
    sub_list_block_num = np.array(sub_list_block_num)
    sub_list_corr = np.array(sub_list_corr)
    sub_list_stim_ctr = np.array(sub_list_stim_ctr)

    sub_data = np.stack([sub_list_sub_id, sub_list_block_num, sub_list_stim, sub_list_actions, sub_list_reward, sub_list_corr, sub_list_stim_ctr, sub_list_set_size], axis=1)
    data = pd.DataFrame(sub_data, columns=['subj_idx', 'block_num', 'stim', 'response', 'feedback', 'corr', 'stim_ctr', 'set_size'])

    return data

In [134]:
model_rl = 'RLWM_v2'
config_rl = model_config_rl[model_rl]

In [135]:
s_size = [1, 2, 3, 4, 5, 6]
freq = [3, 4, 6, 3, 3, 3]

set_size_list = []
for i in range(len(s_size)):
    set_size_list.extend([s_size[i]]*freq[i])

np.random.shuffle(set_size_list)
set_size_list

[1, 2, 1, 2, 3, 3, 4, 3, 5, 3, 4, 5, 2, 3, 3, 2, 1, 4, 5, 6, 6, 6]

In [136]:
num_rep_per_block = [15]

num_datasets = 1
num_actions = 3
subjs = 20

In [137]:
priors_rl_alpha = [0, 0.01]
priors_rl_phi = [0, 0.40]
priors_rl_rho = [0.75, 1]
priors_rl_gamma = [0.60, 0.90]
priors_rl_epsilon = [0, 0.05]
priors_rl_C = [2, 5]

rl_alpha = np.random.uniform(priors_rl_alpha[0], priors_rl_alpha[1], subjs)
rl_phi = np.random.uniform(priors_rl_phi[0], priors_rl_phi[1], subjs)
rl_rho = np.random.uniform(priors_rl_rho[0], priors_rl_rho[1], subjs)
rl_gamma = np.random.uniform(priors_rl_gamma[0], priors_rl_gamma[1], subjs)
rl_epsilon = np.random.uniform(priors_rl_epsilon[0], priors_rl_epsilon[1], subjs)
rl_C = np.random.uniform(priors_rl_C[0], priors_rl_C[1], subjs)

In [138]:
file = list()

for n in range(num_datasets):
    print("\n==> dataset: ", n)
    dataset_file = {}

    dataset_file['info'] = {
                        'num_datasets': num_datasets, 'num_subj': subjs, 'num_actions': num_actions,
                        'model_rl': model_rl,
                        'set_size_list': set_size_list, 'num_rep_per_block': num_rep_per_block
                        }

    dataset_file['data'] = list()
    for i in range(subjs):
        print("[dataset: %d] generating subj data: %d" % (n, i))

        subj_param_rl = np.array([rl_alpha[i], rl_phi[i], rl_rho[i], rl_gamma[i], rl_epsilon[i], rl_C[i]])

        subj_data = {}

        sim_data = simulate_RLWM(subj_param_rl, set_size_list, num_rep_per_block, num_actions=num_actions, subjs=1)
        sim_data['subj_idx'] = i

        subj_data['subj_idx'] = i
        subj_data['true_param'] = subj_param_rl
        subj_data['sim_data'] = sim_data

        #print("\t --", subj_param_rl, subj_data['true_param'])
        dataset_file['data'].append(subj_data)

    file.append(dataset_file)


==> dataset:  0
[dataset: 0] generating subj data: 0
[dataset: 0] generating subj data: 1
[dataset: 0] generating subj data: 2
[dataset: 0] generating subj data: 3
[dataset: 0] generating subj data: 4
[dataset: 0] generating subj data: 5
[dataset: 0] generating subj data: 6
[dataset: 0] generating subj data: 7
[dataset: 0] generating subj data: 8
[dataset: 0] generating subj data: 9
[dataset: 0] generating subj data: 10
[dataset: 0] generating subj data: 11
[dataset: 0] generating subj data: 12
[dataset: 0] generating subj data: 13
[dataset: 0] generating subj data: 14
[dataset: 0] generating subj data: 15
[dataset: 0] generating subj data: 16
[dataset: 0] generating subj data: 17
[dataset: 0] generating subj data: 18
[dataset: 0] generating subj data: 19


In [139]:
dataset = file[0]['data']

In [140]:
list_sub_data = list()
for itr in range(len(dataset)):
    data = dataset[itr]['sim_data']
    data['subj_idx'] = itr

    list_sub_data.append(data)

PR_data = pd.concat(list_sub_data, ignore_index=True)

In [141]:
PR_data

,subj_idx,block_num,stim,response,feedback,corr,stim_ctr,set_size
0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0,0.0,0.0,2.0,1.0,1.0,2.0,1.0
2,0,0.0,0.0,2.0,1.0,1.0,3.0,1.0
3,0,0.0,0.0,2.0,1.0,1.0,4.0,1.0
4,0,0.0,0.0,2.0,1.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...
22195,19,21.0,2.0,2.0,1.0,1.0,15.0,6.0
22196,19,21.0,5.0,1.0,1.0,1.0,15.0,6.0
22197,19,21.0,1.0,0.0,1.0,1.0,15.0,6.0
22198,19,21.0,3.0,0.0,1.0,1.0,15.0,6.0


##PyMC

###Reorganize data for vectorization

In [142]:
df = PR_data.sort_values(['subj_idx', 'block_num', 'stim'])

In [152]:
n_trials = df[['subj_idx','block_num','stim']].value_counts().max()
n_participants = df['subj_idx'].max() + 1
n_choices = int(df['response'].max()) + 1
n_blocks = int(df['block_num'].max()) + 1
n_stimuli = int(df['stim'].max()) + 1
shape = (n_trials, n_participants, n_choices, n_stimuli, n_blocks)
choices = np.ones(shape) * -np.inf
rewards = np.ones(shape) * -np.inf
set_sizes = np.zeros(shape)

for subj_idx in df['subj_idx'].unique():
    for block_num in df['block_num'].unique():
        subj_block_data = df[(df['subj_idx'] == subj_idx) & (df['block_num'] == block_num)]
        subj_block_choices = subj_block_data['response'].to_numpy(dtype=np.int32).reshape((-1, n_trials)).transpose((1,0))
        subj_block_rewards = subj_block_data['feedback'].to_numpy(dtype=np.float32).reshape((-1, n_trials)).transpose((1,0))
        subj_block_set_size = subj_block_data['set_size'].to_numpy(dtype=np.float32).reshape((-1, n_trials)).transpose((1,0))
        n_stimuli = int(subj_block_data['set_size'].max())
        subj_block_choices = np.eye(n_choices)[subj_block_choices].transpose((0,2,1))
        subj_block_rewards = subj_block_rewards.reshape((n_trials, 1, n_stimuli)).repeat(n_choices, axis=1)
        subj_block_set_size = subj_block_set_size.reshape((n_trials, 1, n_stimuli)).repeat(n_choices, axis=1)
        choices[:, subj_idx, :, :n_stimuli, int(block_num)] = subj_block_choices
        rewards[:, subj_idx, :, :n_stimuli, int(block_num)] = subj_block_choices * subj_block_rewards
        set_sizes[:, subj_idx, :, :n_stimuli, int(block_num)] = subj_block_set_size

###Prep for PyMC

In [153]:
df_model = df.sort_values(['block_num', 'stim', 'subj_idx'])

# Make coordinates
participant_ids, participants = pd.factorize(df_model['subj_idx'])
coords = {
          "participant_idx": np.array(participants),
          # "participant_idx": np.arange(len(participant_ids)),
          "obs_idx": np.arange(df_model.shape[0])
         }

In [154]:
def rlwm_hdll(pA_, pB_, pC_, pE_, pG_, pP_, pR_, choices, rewards, set_sizes_):
    dq_RL_ = np.ones(shape)[0] * 1.0 / n_choices
    dq_WM_ = np.ones(shape)[0] * 1.0 / n_choices
    dC_ = choices
    dR_ = rewards

    return rlwm_recovery(dq_RL_, dq_WM_, dC_, dR_, pA_, pB_, pC_, pE_, pG_, pP_, pR_, set_sizes_).flatten()

###Run PyMC

In [158]:
with pm.Model(coords=coords) as hierarchical_model:
    # Data
    Choices = pm.ConstantData('Choices', choices)
    Rewards = pm.ConstantData('Rewards', rewards)
    Set_sizes = pm.ConstantData('Set_sizes', set_sizes)


    # Hyperpriors for group nodes
    pA_alpha = pm.Gamma("pA_alpha", alpha = 3. , beta = 3.)
    pA_beta = pm.Gamma("pA_beta", alpha = 3. , beta = 3.)
    pB_alpha = pm.Gamma("pB_alpha", alpha = 3. , beta = 3.)
    pB_beta = pm.Gamma("pB_beta", alpha = 3. , beta = 3.)
    pC_alpha = pm.Gamma("pC_alpha", alpha = 3. , beta = 3.)
    pC_beta = pm.Gamma("pC_beta", alpha = 3. , beta = 3.)
    pE_alpha = pm.Gamma("pE_alpha", alpha = 3. , beta = 3.)
    pE_beta = pm.Gamma("pE_beta", alpha = 3. , beta = 3.)
    pG_alpha = pm.Gamma("pG_alpha", alpha = 3. , beta = 3.)
    pG_beta = pm.Gamma("pG_beta", alpha = 3. , beta = 3.)
    pP_alpha = pm.Gamma("pP_alpha", alpha = 3. , beta = 3.)
    pP_beta = pm.Gamma("pP_beta", alpha = 3. , beta = 3.)
    pR_alpha = pm.Gamma("pR_alpha", alpha = 3. , beta = 3.)
    pR_beta = pm.Gamma("pR_beta", alpha = 3. , beta = 3.)


    # Priors for alphas and betas
    pA = pm.Beta(name="pA", alpha=pA_alpha, beta=pA_beta, dims="participant_idx")
    pB = pm.Beta(name="pB", alpha=pB_alpha, beta=pB_beta, dims="participant_idx")
    pC = pm.Beta(name="pC", alpha=pC_alpha, beta=pC_beta, dims="participant_idx")
    pE = pm.Beta(name="pE", alpha=pE_alpha, beta=pE_beta, dims="participant_idx")
    pG = pm.Beta(name="pG", alpha=pG_alpha, beta=pG_beta, dims="participant_idx")
    pP = pm.Beta(name="pP", alpha=pP_alpha, beta=pP_beta, dims="participant_idx")
    pR = pm.Beta(name="pR", alpha=pR_alpha, beta=pR_beta, dims="participant_idx")


    # Render parameters trial wise
    pA_trial = pA[participant_ids]
    pB_trial = pB[participant_ids]
    pC_trial = pC[participant_ids]
    pE_trial = pE[participant_ids]
    pG_trial = pG[participant_ids]
    pP_trial = pP[participant_ids]
    pR_trial = pR[participant_ids]


    # CHOOSE WHICH WRAPPER FUNCTION TO USE HERE
    pm.Potential("likelihood", rlwm_hdll(pA_trial,pB_trial,pC_trial,pE_trial,pG_trial,pP_trial,pR_trial,Choices,Rewards,Set_sizes))
    idata_pooled = pm.sample(500, tune=500, chains=1, nuts_sampler="numpyro")

/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/shape.py:158: UserWarning: Failed to infer_shape from Op Mul.
Input shapes: [(TensorConstant(TensorType(int64, shape=()), data=array(15)), TensorConstant(TensorType(int64, shape=()), data=array(20)), TensorConstant(TensorType(int64, shape=()), data=array(3)), TensorConstant(TensorType(int64, shape=()), data=array(6)), TensorConstant(TensorType(int64, shape=()), data=array(22))), (TensorConstant(TensorType(int64, shape=()), data=array(1)), TensorConstant(TensorType(int64, shape=()), data=array(1)), TensorConstant(TensorType(int64, shape=()), data=array(1)), TensorConstant(TensorType(int64, shape=()), data=array(1)), TensorConstant(TensorType(int64, shape=()), data=array(22200)))]
Exception encountered during infer_shape: <class 'ValueError'>
Exception message: Could not broadcast dimensions. Incompatible shapes were [(TensorConstant(TensorType(int64, shape=()), data=array(15)), TensorConstant(TensorType(int64, shape=()), 

KeyboardInterrupt: 